<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q keras

In [18]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation filekk
path_to_data = 'fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:192000]


In [21]:
print(pairs[116999])

['genetically-modified organisms', 'organismes génétiquement modifiés']


In [ ]:
def clean_sentence(sentence):
    
    lower_case_sent = sentence.lower()
  
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [15]:
from Book1 import words 
print(words[0])
# for word in words:
#   z=word['English']+'\t'+word['French']
# #   raw_data3.append(z)
# text_file = open("sample.txt", "wt")
with open('fra_eng_sc.txt', 'w') as f:
  for word in words:
    z=word['English'].split('\/')[0]+'\t'+word['French'].split('\/')[0]
    f.write("%s\n" % z)

f.close()


{'English': 'arrange', 'French': 'organiser', 'Arabic': 'نظم'}
